In [1]:
import os
import numpy as np 
from PIL import Image
from skimage import measure 

def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))

            # If the pixel is not black...
            if pixel != (0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks

In [21]:
from shapely.geometry import Polygon, MultiPolygon # (pip install Shapely)

def create_sub_mask_annotation(image_name, sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5, positive_orientation='low')
    
    segmentations = []
    polygons = []
    
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)
        
    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }
    
    image = {
        'licence': 0,
        'file_name': image_name,
        'width': sub_mask.size[0] - 2,
        'height': sub_mask.size[1] - 2,
        'id': annotation_id
    }

    return annotation, image

In [34]:
import json
from tqdm import tqdm

dataset_dir = '/home/vlados/Documents/Mask_RCNN/generated/train'
masks_names = sorted(os.listdir(os.path.join(dataset_dir, 'masks')))
images_names = sorted(os.listdir(os.path.join(dataset_dir, 'images')))

# Define which colors match which categories in the images
tail_id = 1

category_ids = { 1: {'255': tail_id} }
category_id = 1
# These ids will be automatically increased as we go
image_id = 0
is_crowd = 0
annotation_id = 0

# Create the annotations
images = []
annotations = []

for i in tqdm(range(len(masks_names))):
    tail_mask = Image.open(os.path.join(dataset_dir, 'masks', masks_names[i]))
    sub_masks = create_sub_masks(tail_mask)
    for color, sub_mask in sub_masks.items():
        #category_id = category_ids[image_id][color]
        annotation, image = create_sub_mask_annotation(images_names[i], sub_mask, image_id, 
                                                category_id, annotation_id, is_crowd)
        images.append(image)
        annotations.append(annotation)
        annotation_id += 1
    image_id += 1

supercategory = [{"supercategory": "humpback", "id": 1, "name": "teils"}]

with open('/home/vlados/Documents/Mask_RCNN/coco_annotations.json', 'w') as outfile:
    json.dump({'images': images, 'annotations': annotations, 'categories': supercategory}, outfile)

100%|██████████| 2000/2000 [14:30<00:00,  2.81it/s]
